In [1]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [8]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [9]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [10]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [13]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [14]:
x_train.head()

,age,gender,fever,cough,city
24,13,Female,100.0,Strong,Kolkata
12,25,Female,99.0,Strong,Kolkata
17,40,Female,98.0,Strong,Delhi
94,79,Male,NaN,Strong,Kolkata
81,65,Male,99.0,Mild,Delhi


In [18]:
#handmissing value for fever

si=SimpleImputer()
x_train_fever=si.fit_transform(x_train[['fever']])

x_test_fever=si.fit_transform(x_test[['fever']])

In [22]:
x_train_fever.shape

(80, 1)

In [23]:
#ordinal encoding

oe=OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough=oe.fit_transform(x_train[['cough']])

x_test_cough=oe.fit_transform(x_test[['cough']])


In [27]:
#onehotencoding

hoe=OneHotEncoder(drop='first',sparse=False)
x_train_gender_city=hoe.fit_transform(x_train[['gender','city']])

x_test_gender_city=hoe.fit_transform(x_test[['gender','city']])

/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [29]:
x_train_gender_city.shape

(80, 4)

In [31]:
# Extracting Age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [32]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
# also the test data
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

x_train_transformed.shape

(80, 7)

## Column Transformer

In [33]:
from sklearn.compose import ColumnTransformer

In [34]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [36]:
transformer.fit_transform(x_train).shape

/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)